<a href="https://colab.research.google.com/github/MohaMAZ/Webscraping/blob/main/web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
import string
import pandas as pd
import string
import re

In [ ]:
#Since the website offers the option to search medicines by the starter alphabet I have to iterate over all the 26 alphabets
for k in list(string.ascii_uppercase):
  #creating the dataframe that I want to store data in
  df = pd.DataFrame(columns = ['Prefix', 'Name', 'Price'])
  #define the target url
  my_url = "https://medicament.ma/listing-des-medicaments/?lettre="+k
  client = uReq(my_url)
  page_html = client.read()
  client.close()
  page_soup = soup(page_html)
  #page_soup.findAll('strong')[0].text used to find the number of medicines that start with a specific alphabet.
  #And I divided that number by 20 since every page contains 20 medicines maximum.
  #So in case the number of medicines <20 , let's say 19 the result of int(19/20) will be 0 so added that +1 o fix that.
  #other scenario : In case the number of medicines is 20 the result of (20/20)+1 will be 2 , the solution:
  #I substracted -0.5 (you can use any float number x so that -1<x<0) so the result will be 1.
  #So finnaly we got the j (number of pages related to each alphabet) so we can loop over it
  j = int((int(page_soup.findAll('strong')[0].text)-0.5)/20)+1
  for i in range(1,j+1):
    #loop over pages of a specific letter
    my_url = "https://medicament.ma/listing-des-medicaments/page/"+str(i)+"/?lettre="+k
    client = uReq(my_url)
    page_html = client.read()
    client.close()
    page_soup = soup(page_html)
    details = page_soup.findAll('span',{"class" : "details"})
    for item in details :
      res = re.split("\n",item.text)
      try:
        df = df.append({'Prefix' : k , 'Name' : res[1].strip(), 'Price' : re.split(' ',re.split(": ",res[2])[1])[0]}, 
                    ignore_index = True)
      except:
        df = df.append({'Prefix' : k , 'Name' : item.text, 'Price' : 'Null'}, 
                      ignore_index = True)
  #save every alphabet medicines into csv file
  df.to_csv(k+"_data.csv",index=False)